(omero-tools:synchronize-tags)=

# Synchronize tags across groups

To make sure that there is some standardization across projects with regard to how data is tagged and annotated, we need to synchronize tags across groups. The goal is to make sure that the tags used in the data are consistent across groups. This will make it easier to search for data and to use the data in the future. The defined tags themselves are based off of the [REMBI metadata schema](https://www.nature.com/articles/s41592-021-01166-8). More specifically, we orient ourselves on the tags used in the [provided spreadsheet](https://docs.google.com/spreadsheets/d/1Ck1NeLp-ZN4eMGdNYo2nV6KLEdSfN6oQBKnnWU6Npeo/edit).


To install the necessary packages, run the following command:
```bash
mamba install omero-py ezomero -c bioconda
```

In [ ]:
from omero.gateway import BlitzGateway
import omero
import omero.gateway
from omero.rtypes import rstring, rlong
import yaml
import ezomero
import yaml


# Prerequisites

We create user that is called `tag-man` and make sure that this user is member of all groups to which we want to synchronize tags. We then add these groups to the groups we want to synchronize.

In [6]:
# Replace these with your OMERO server credentials
OMERO_HOST = 'omero-int.biotec.tu-dresden.de'
OMERO_PORT = 4064
OMERO_USER = 'tag-man'
OMERO_PASS =  # add tag-man password here
OMERO_SECURE = True
TAG_SET_NAMESPACE = 'openmicroscopy.org/omero/insight/tagset'

TARGET_GROUPS = ['BiAPoL', 'Campas Lab', 'Dye Lab']

We need some utility functions to read tags from a tagset and to write tags to a tagset. We also need a function to synchronize tags across groups.

In [3]:
def load_yaml(file_path):
    with open(file_path, 'r') as file:
        return yaml.safe_load(file)

def get_user_tag_sets(conn, user_id, namespace):
    params = omero.sys.ParametersI()
    params.add('uid', rlong(user_id))
    params.add('ns', rstring(namespace))

    query = """
    select t from TagAnnotation t
    join fetch t.details.owner as owner
    where owner.id = :uid and t.ns = :ns
    """

    tag_sets = conn.getQueryService().findAllByQuery(query, params)
    return {tag_set.textValue.val: tag_set for tag_set in tag_sets}

def create_tag_set(conn, name, description, user_id, namespace):
    tag_set = omero.model.TagAnnotationI()
    tag_set.textValue = rstring(name)
    tag_set.description = rstring(description)
    tag_set.ns = rstring(namespace)
    tag_set.details.owner = omero.model.ExperimenterI(user_id, False)
    return conn.getUpdateService().saveAndReturnObject(tag_set)

def list_tags_in_tagset(conn, tagset_name, user_id, namespace):
    params = omero.sys.ParametersI()
    params.add('uid', rlong(user_id))
    params.add('ns', rstring(namespace))
    params.add('name', rstring(tagset_name))

    query = """
    select t from TagAnnotation t
    join fetch t.details.owner as owner
    where owner.id = :uid and t.ns = :ns and t.textValue = :name
    """

    tag_sets = conn.getQueryService().findAllByQuery(query, params)
    
    if not tag_sets:
        print(f"No tag set found with the name '{tagset_name}'.")
        return []

    tag_set = tag_sets[0]

    # Load annotations linked to the tag set
    links = conn.getAnnotationLinks('TagAnnotation', [tag_set.id.val])
    tag_names = [link.child.textValue.val for link in links]

    return tag_names

Lastly, this function will be called and iterates over every group that is part of the synchronization process (see above) and synchronizes the tags. For this, it checks existing tags and tagsets. If there is a tag that exists in the [local collection of tags](tag_collection.yaml) but doesn't exist in the remote tagset, it will be added. If a tag exists remotely but not locally in the preset collection, nothing happens. This makes sure that users can add their own individual tags to the tagmanager without them being overwritten by the synchronization process.

In [4]:
def synchronize_tags(conn, local_tag_collection):

    # Get the currently connected user's ID
    user_id = conn.getUserId()
    print(f"Currently connected user's ID: {user_id}")

    # Process each tag set
    existing_tag_sets = get_user_tag_sets(conn, user_id, TAG_SET_NAMESPACE)

    for tagset in local_tag_collection:
        tagset_name = local_tag_collection[tagset]['name']
        tagset_description = local_tag_collection[tagset]['description']
        
        # Check if tag set exists in OMERO
        if tagset_name in existing_tag_sets:
            print(f"Tag set '{tagset_name}' already exists in OMERO.")
            tag_set = existing_tag_sets[tagset_name]
        else:
            print(f"Creating tag set '{tagset_name}' in OMERO.")
            tag_set = create_tag_set(conn, tagset_name, tagset_description, user_id, TAG_SET_NAMESPACE)

        tags_in_tagset = list_tags_in_tagset(conn, tagset_name, user_id, TAG_SET_NAMESPACE)

        if local_tag_collection[tagset]['tags'] is None:
            continue
        
        for tag in local_tag_collection[tagset]['tags']:    
            tag_name = tag['name']
            tag_description = tag['description']
            
            if tag_name in tags_in_tagset:
                print(f"\tTag '{tag_name}' already exists in tag set '{tagset_name}'.")
            else:
                print(f"\tCreating tag '{tag_name}' in tag set '{tagset_name}'.")
                tag_annotation = omero.model.TagAnnotationI()
                tag_annotation.textValue = rstring(tag_name)
                tag_annotation.description = rstring(tag_description)
                tag_annotation.details.owner = omero.model.ExperimenterI(user_id, False)
                tag_annotation = conn.getUpdateService().saveAndReturnObject(tag_annotation)

                # Link the new tag to the tag set
                link = omero.model.AnnotationAnnotationLinkI()
                link.parent = omero.model.TagAnnotationI(tag_set.id.val, False)
                link.child = omero.model.TagAnnotationI(tag_annotation.id.val, False)
                conn.getUpdateService().saveAndReturnObject(link)
                tag_annotation.details.owner = omero.model.ExperimenterI(user_id, False)
                tag_annotation = conn.getUpdateService().saveAndReturnObject(tag_annotation)

                # Link the new tag to the tag set
                link = omero.model.AnnotationAnnotationLinkI()
                link.parent = omero.model.TagAnnotationI(tag_set.id.val, False)
                link.child = omero.model.TagAnnotationI(tag_annotation.id.val, False)
                #conn.getUpdateService().saveAndReturnObject(link)

Let's show all the tagsets and tags in our local collection and then synchronize them across groups.

In [7]:
# Load the YAML document
yaml_file_path = './tag_collection.yaml'  # Replace with the actual path to your YAML file
local_tag_collection = load_yaml(yaml_file_path)


for OMERO_GROUP in TARGET_GROUPS:
    # Connect to OMERO
    conn = ezomero.connect(user=OMERO_USER, password=OMERO_PASS, host=OMERO_HOST, port=OMERO_PORT, secure=True, group=OMERO_GROUP)
    synchronize_tags(conn, local_tag_collection)

Currently connected user's ID: 212
Tag set 'Biological entity' already exists in OMERO.
	Tag 'Brain' already exists in tag set 'Biological entity'.
	Tag 'Cell Culture' already exists in tag set 'Biological entity'.
	Tag 'Cells' already exists in tag set 'Biological entity'.
	Tag 'Embryo' already exists in tag set 'Biological entity'.
	Tag 'Eye' already exists in tag set 'Biological entity'.
	Tag 'Fin' already exists in tag set 'Biological entity'.
	Tag 'Gastro-intestinal' already exists in tag set 'Biological entity'.
	Tag 'Organoid' already exists in tag set 'Biological entity'.
	Tag 'Spheroid' already exists in tag set 'Biological entity'.
	Tag 'Tail' already exists in tag set 'Biological entity'.
	Tag 'Whole organism' already exists in tag set 'Biological entity'.
	Tag 'Wing Disk' already exists in tag set 'Biological entity'.
Tag set 'Organism' already exists in OMERO.
	Tag 'C. elegans' already exists in tag set 'Organism'.
	Tag 'Drosophila' already exists in tag set 'Organism'.
	T